## Load the Raygun model and embedding

In [1]:
from raygun.pretrained import raygun_2_2mil_800M
# 
raymodel = raygun_2_2mil_800M()
raymodel = raymodel.to(0)

# esm-2 model
from esm.pretrained import esm2_t33_650M_UR50D
model, alph = esm2_t33_650M_UR50D()
model       = model.to(0)

Using cache found in /hpc/home/kd312/.cache/torch/hub/rohitsinghlab_raygun_main


## Initialize the dataloader

In [2]:
from tqdm import tqdm
from torch.utils.data import DataLoader
from raygun.modelv2.loader import RaygunData

preddata   = RaygunData("../data/fastas/all_gfp_seqs.fasta", 
                        alph, model, device = 0)
predloader = DataLoader(preddata, shuffle = True, 
                       batch_size = 3, collate_fn=preddata.collatefn) ## use the collatefn provided in RaygunData
len(preddata)

933

## Get one batch, run it through Raygun and get fixed-length embeddings

In [3]:
toks, embs, mask, bat = next(iter(predloader))
toks = toks.to(0)
embs = embs.to(0)
mask = mask.to(0)

In [5]:
results = raymodel(embs, mask=mask, noise = 0.0, 
                  return_logits_and_seqs = False)

## Obtain the fixed length representation

Should have the dimension: batch_size, 50, 1280
regardless of the original sequence length

In [6]:
flemb = results["fixed_length_embedding"]
flemb.shape

torch.Size([3, 50, 1280])